# 5. MalConv 전이학습

* 필요한 package 설치 확인 및 적용

In [ ]:
# !pip install torch
# !pip install scikit-learn

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

from MalConv import MalConv
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

## 1. 학습을 위한 도구 구축

In [ ]:
### Stream data csv 추출 도구 ###
def parse_data(data_path):
    print("### START PARSING DATA ###")
    with open(data_path, "r", encoding="cp949") as data:
        label = []
        stream_data = []
        for idx, line in enumerate(data.readlines()):
            try:
                if idx != 0 and len(line) < 100000:
                    look = line.rfind("]")
                    line = line[look + 2 :]
                    line = line.replace("\n", "")
                    if line.startswith(","):
                        line = line[1:]
                    line = line.split(",")
                    line = [int(x, 0) for x in line]

                    if line[1] == 0:
                        label.append(0)
                    else:
                        label.append(1)
                    stream_data.append(line[4:])
            except:
                pass
        data.close()
    print("### PARSING DONE! ###")
    return stream_data, label

In [ ]:
### 학습을 위한 Dataset Class 구축 ###

class StreamDataset(Dataset):
    """
    Stream data를 학습에 이용할 수 있도록 Dataset class 제작
    """
    def __init__(self, stream_list, label):
        """
        :param stream_list: csv에서 추출한 stream list
        :param label: 악성/정상 여부
        """
        # 1. 기존 Dataset 상속
        # 2. stream_list, label 변수 지정
        self.stream_list = stream_list
        self.label = label

    def __len__(self):
        return len(self.label)

    def __getitem__(self, index):
        return self.stream_list[index], self.label[index] 

In [ ]:
### 데이터셋 내 일정한 길이의 stream 으로 만들기 위한 함수 ###

def collate_fn(batch, max_len=100000):
    """
    :param int batch: batch로 구성되어 있는 데이터셋
    :param int max_len: 학습에 이용할 stream의 최대 길이
    """
    # TODO
    # 1. stream_list, label_list 변수 선언
    # 2. batch 내의 stream, label 학습에 맞는 변형
    # 3. max_len 초과의 길이의 경우 max_len 만큼만 자르기
    # 4. max_len 미만의 길이의 경우 padding(0) 추가하여 길이 맞추기
    # 5. stream은 long 형태, label은 int64 형태로 list에 삽입
    # 6. batch에 맞는 stack 맞춰서 return

    stream_list = []
    label_list = []

    for stream, label in batch:
        if len(stream) < max_len:
            stream += [0] * (max_len - len(stream))
        stream_processed = torch.tensor(stream[:max_len], dtype=torch.long)
        label_processed = torch.tensor(label, dtype=torch.int64)
        stream_list.append(stream_processed)
        label_list.append(label_processed)
    stream_list = pad_sequence(stream_list, batch_first=True, padding_value=0)

    return stream_list, torch.stack(label_list)    

## 2. 학습 함수 구축

In [ ]:
def train(model, train_dataloader, loss_fn, optimizer, device):
    """
    매 epoch 별 학습을 위한 함수
    :param nn.Module model: 학습 대상 모델
    :param DataLoader train_dataloader: 학습 dataloader
    :param nn.Module loss_fn: 학습에 사용할 loss 산정 함수
    :param nn.Module optimizer: 학습 paramter 조정 함수
    :param device: 학습 장치
    """
    # TODO
    ### Epoch별 학습 함수 구축 ###
    # 1. model train 상태 선언
    # 2. 기록용 loss, 정답 수, 현재까지의 평가한 데이터 수 변수 선언
    # 3. 배치 별 stream과 정답 학습장치 할당
    # 4. 학습을 위한 optimizer 초기화
    # 5. 모델을 이용한 예측된 정답 백터 도출
    # 6. 실제 정답과 예측된 답의 loss 계산
    # 7. loss에 기반한 역전파 연산 진행
    # 8. 학습 optimizer step
    # 9. 가장 확률이 높은 label 추출
    # 10. batch 내 정답과 일치한 예측 수 확인
    # 11. 현재까지 학습한 데이터 개수 더하기

    model.train()

    cur_loss = 0
    correct = 0
    counts = 0

    for idx, (stream, label) in enumerate (train_dataloader):
        stream = stream.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(stream)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        output = output.argmax(dim=1)

        correct += (output == label).sum().item()
        counts += len(label)

        cur_loss += loss.item()
    print(
        f"Training loss {cur_loss / (idx+1) : .5f}, Traiing accuracy {correct / counts: 5f}"
    )


## 3. 평가 함수 구축

In [ ]:
def eval(model, eval_dataloader, loss_fn, device):
    """
    :param nn.Module model: 평가에 사용할 모델
    :param DataLoader eval_dataloader: 평가 데이터셋 dataloader
    :param nn.Module loss_fn: loss 계산에 사용할 함수
    :param device: 평가에 사용할 장치
    """
    #TODO
    ### 평가 함수 구축 ###
    # 1. model 평가과정 선언
    # 2. torch 초기화 선언
    # 3. 정답율 확인을 위한 변수 및 loss 저장용 변수 선언
    # 4. 평가 데이터의 stream data 및 정답 평가 장치에 할당
    # 5. 모델을 통한 예측된 정답 도출
    # 6. 예측된 정답과 실제 정답 사이 loss 산출
    # 7. 일치한 정답 수 계산
    # 8. 현재 loss 변수 저장
    # 9. 전체 평균 loss 및 accuracy 계산

    model.eval()

    with torch.no_grad():
        correct = 0
        curr_loss = 0

        for stream, label in eval_dataloader:
            stream = stream.to(device)
            label = label.to(device)
            output = model(stream)
            loss = loss_fn(output, label)
            output = output.argmax(dim=1)
            correct += (output == label).sum().item()
            curr_loss += loss.item()

    accuracy = correct / len(eval_dataloader.dataset)
    loss_result = curr_loss / len(eval_dataloader)

    return loss_result, accuracy

## 4. 전이학습 과정 구축

In [ ]:
DATA_PATH = ""

In [ ]:
### 전이학습 과정 구축 ###
# 1. stream data csv의 데이터 추출
# 2. 학습/평가를 위한 데이터 분리 (sklearn 이용)
# 3. 학습/평가 데이터 Dataset 구성
# 4. 학습 평가 Dataset for loop을 위한 DataLoader 구성
# 5. 학습 장치 설정
# 6. MalConv 모델 선언 및 학습된 weight load 및 설정



stream, label = parse_data(DATA_PATH)

train_stream, valid_stream, train_label, valid_label = train_test_split(
    stream, label, test_size=0.2, shuffle=False
)

train_dataset = StreamDataset(train_stream, train_label)
valid_dataset = StreamDataset(valid_stream, valid_label)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=lambda x: collate_fn(x),
)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=lambda x: collate_fn(x),
)
    
device = (
    torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)

print(f"Current device is {device}")

model = MalConv(channels=256, window_size=512, embd_size=8)
weight = torch.load("../data/malconv.checkpoint")
model.load_state_dict(weight["model_state_dict"])

# TODO
# 7. FC layer 학습을 위한 FC layer 초기화
# 8. loss 함수 및 optimizer 설정
# 9. model 학습장치 할당
# 10. 최소 loss 저장용 변수 설정
# 11. 매 epoch 별 학습 진행
# 12. 매 epoch 별 평가 진행
# 13. epoch 별 loss 비교 후 최소 loss 설정 및 최소 loss 모델 저장


model.fc_1 = nn.Linear(in_features=256, out_features=256, bias=True)
model.fc_2 = nn.Linear(in_features=256, out_features=2, bias=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
model.to(device)

min_loss = 0

for epoch in range(1, 11):
    print(f"### Train Epoch {epoch} ###")
    train_loss, train_accuracy = train(
        model, train_dataloader, loss_fn, optimizer, device
    )
    val_loss, val_accuracy = eval(model, valid_dataloader, loss_fn, device)

    print(
        f"### Epoch {epoch}, train_loss: {train_loss}, train_accuracy: {train_accuracy}, val_loss: {val_loss}, val_accuracy: {val_accuracy} ###"
    )

    if epoch == 1:
        print("Saving Initial model")
        min_loss = val_loss
        torch.save(
            model.state_dict(),
            "./malconv_transfer_learning.pth",
        )
    else:
        if val_loss < min_loss:
            print("Loss has been improved! Save model")
            min_loss = val_loss
            torch.save(
                model.state_dict(),
                "./malconv_transfer_learning.pth",
            )
print("Training Finish")
